# yelp data ingestion

In [1]:
from pymongo import MongoClient
from bson import json_util
import pymongo
import pprint
import boto3
import json
import bson
import sys
import os

pp = pprint.PrettyPrinter(indent = 3)
print('imported modules.')

imported modules.


## connect to s3

In [2]:
client = boto3.client('s3')

bucket_meta = client.list_objects(Bucket = 'yelp-academic-dataset-123')
print('files in s3 bucket:')
print('')
for c in bucket_meta['Contents']:
    print(c['Key'])

files in s3 bucket:

yelp_academic_dataset_business.json
yelp_academic_dataset_checkin.json
yelp_academic_dataset_review.json
yelp_academic_dataset_tip.json
yelp_academic_dataset_user.json


## function to read json from s3

In [4]:
def read_json(filename):
    """
    reads a yelp .json file from s3 bucket.

    keyword arguments:
    filename - name of file (str)

    returns: json_file (json)
    """

    response = client.get_object(Bucket = 'yelp-academic-dataset-123', Key = filename)
    file_content = response['Body'].read().decode('utf-8')
    json_file = json_util.loads("[" + file_content.replace("}\n{", "},\n{") + "]")
    return(json_file)

## load json to mongodb

mongodb issues: can't upload more than 16mb to a collection at a time, "fully qualified namespace too long" (length of field in json record).

In [13]:
def mongo_upload(endpoint):
    """
    uploads json object to mongodb collection

    keyword arguments:
    endpoint - name of endpoint (json)

    returns: None
    """
    try:
        with open('mongo_creds.json') as f:
            mongo_creds = json.load(f)

    except:
        print('no credentials found.')

    connect_str = "mongodb://" + mongo_creds['User'] + ":" + mongo_creds['Password'] + "@" + mongo_creds['Host'] + ":" + str(mongo_creds['Port'])
    myclient = MongoClient(connect_str)  
    db = myclient["yelp"]
    endpoint_str = str(endpoint)
    Collection = db[endpoint_str] 
    Collection.insert_many(endpoint)   
    # Collection.insert_one(endpoint) 
    # print('uploaded {} collection to mongodb.'.format(endpoint_str))

## business

In [46]:
business = read_json('yelp_academic_dataset_business.json')
print('read in json file.')

read in json file.


In [47]:
print('json file has {} records with size of {} mb.'.format(len(business), sys.getsizeof(business)/1000000))
print('here is an example record:')
print('')
pp.pprint(business[0])

json file has 160585 records with size of 1.443576 gb.
here is an example record:

{  'address': '921 Pearl St',
   'attributes': {  'Alcohol': "'beer_and_wine'",
                    'Ambience': "{'touristy': False, 'hipster': False, "
                                "'romantic': False, 'divey': False, "
                                "'intimate': False, 'trendy': False, "
                                "'upscale': False, 'classy': False, 'casual': "
                                'True}',
                    'BikeParking': 'True',
                    'BusinessAcceptsBitcoin': 'False',
                    'BusinessAcceptsCreditCards': 'True',
                    'BusinessParking': "{'garage': False, 'street': True, "
                                       "'validated': False, 'lot': False, "
                                       "'valet': False}",
                    'Caters': 'True',
                    'DogsAllowed': 'False',
                    'GoodForMeal': "{'dessert': False,

In [40]:
del business

## checkin

In [11]:
checkin = read_json('yelp_academic_dataset_checkin.json')
print('read in json file.')

read in json file.


In [69]:
print('json file has {} records with size of {} mb.'.format(len(checkin), sys.getsizeof(checkin)/1000000))
print('here is an example record:')
print('')
pp.pprint(checkin[0])

json file has 138876 records with size of 1.140568 mb.
here is an example record:

{  '_id': ObjectId('604eeb3015fde9f88c4d8df3'),
   'business_id': '--0r8K_AQ4FZfLsX3ZYRDA',
   'date': '2017-09-03 17:13:59'}


In [76]:
for i in range(1,len(checkin)):
    mongo_upload(checkin[i])

print('uploaded checkin collection.')

OperationFailure: Fully qualified namespace is too long. Namespace: yelp.{'business_id': '--0zrn43LEaB4jUWTQH_Bg', 'date': '2010-10-08 22:21:20, 2010-11-01 21:29:14, 2010-12-23 22:55:45, 2011-04-08 17:14:59, 2011-04-11 21:28:45, 2011-04-26 16:42:25, 2011-05-20 19:30:57, 2011-05-24 20:02:21, 2011-08-29 19:01:31', '_id': ObjectId('604eeb3015fde9f88c4d8df4')} Max: 255, full error: {'ok': 0.0, 'errmsg': "Fully qualified namespace is too long. Namespace: yelp.{'business_id': '--0zrn43LEaB4jUWTQH_Bg', 'date': '2010-10-08 22:21:20, 2010-11-01 21:29:14, 2010-12-23 22:55:45, 2011-04-08 17:14:59, 2011-04-11 21:28:45, 2011-04-26 16:42:25, 2011-05-20 19:30:57, 2011-05-24 20:02:21, 2011-08-29 19:01:31', '_id': ObjectId('604eeb3015fde9f88c4d8df4')} Max: 255", 'code': 4862100, 'codeName': 'Location4862100'}

In [14]:
mongo_upload(checkin)


DocumentTooLarge: BSON document too large (92 bytes) - the connected server supports BSON document sizes up to 16777216 bytes.

In [16]:
pp.pprint(checkin)

:26, '
              '2020-08-31 18:51:28, 2020-11-23 19:01:21, 2020-11-27 18:11:52, '
              '2020-12-28 20:36:41, 2020-12-28 20:37:52, 2021-01-01 16:51:04'},
   {  '_id': ObjectId('604f9773c4078d81432ffd50'),
      'business_id': 'G_QFvmWWmJRC-bsWC4bYqw',
      'date': '2010-07-21 23:13:40, 2010-08-03 20:16:48, 2011-01-21 23:33:30, '
              '2011-04-26 18:27:07, 2011-07-20 17:31:37, 2011-07-22 19:51:18, '
              '2012-03-25 01:08:27, 2012-04-14 18:34:58, 2013-01-01 14:53:20, '
              '2014-07-26 18:09:26, 2015-03-26 23:26:34, 2015-09-02 12:14:57, '
              '2015-09-03 22:02:55, 2015-09-15 13:06:04, 2015-11-23 12:47:22, '
              '2015-11-30 13:48:01, 2016-02-04 12:34:50, 2016-02-11 12:26:13, '
              '2016-02-17 14:16:28, 2016-02-29 14:30:04, 2016-03-01 12:33:33, '
              '2016-03-16 12:01:27, 2016-03-18 21:57:52, 2016-03-22 11:23:42, '
              '2017-01-09 13:24:04, 2017-01-14 14:24:44, 2017-02-07 12:00:09, '
              '

KeyboardInterrupt: 

In [17]:
del checkin

## tip

In [50]:
tip = read_json('yelp_academic_dataset_tip.json')
print('read in json file.')

read in json file.


In [51]:
print('json file has {} records with size of {} mb.'.format(len(tip), sys.getsizeof(tip)/1000000))
print('here is an example record:')
print('')
pp.pprint(tip[0])

json file has 1162119 records with size of 9.504856 gb.
here is an example record:

{  'business_id': 'ENwBByjpoa5Gg7tKgxqwLg',
   'compliment_count': 0,
   'date': '2011-07-22 19:07:35',
   'text': 'Carne asada chips...',
   'user_id': 'WCjg0jdHXMlwbqS9tZUx8Q'}


In [42]:
del tip

## user

In [5]:
user = read_json('yelp_academic_dataset_user.json')
print('read in json file.')

OverflowError: signed integer is greater than maximum

In [ ]:
print('json file has {} records with size of {} mb.'.format(len(user), sys.getsizeof(user)/1000000))
print('here is an example record:')
print('')
pp.pprint(user[0])

In [ ]:
del user

## review

In [6]:
review = read_json('yelp_academic_dataset_review.json')
print('read in json file.')

OverflowError: signed integer is greater than maximum

In [ ]:
print('json file has {} records with size of {} mb.'.format(len(review), sys.getsizeof(review)/1000000))
print('here is an example record:')
print('')
pp.pprint(review[0])

In [ ]:
del review